Нужно защитить данные клиентов страховой компании. Разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуем корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

**Описание данных**

Признаки: 

    пол
    возраст
    зарплата застрахованного
    количество членов его семьи


Целевой признак: 

    количество страховых выплат клиенту за последние 5 лет

## 1. Загрузка данных

In [ ]:
import pandas as pd
import numpy as np
import seaborn
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, roc_curve, roc_auc_score, accuracy_score
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from scipy import stats as st
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

Загрузим и посмотрим на данные

In [ ]:
data = pd.read_csv('/datasets/insurance.csv')

In [ ]:
data.head(3)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Пропусков в данных нет.

Признаки носят названия на кириллице, для удобства заменим их на аналоги латинницы.

2 признака - возраст и зар.плата имеют тип float, посмотрим критично ли переводить их в тип данных int, для экономии памяти и ускорения рабочего процесса.

In [ ]:
data = data.rename(columns={"Пол": "gender", "Возраст": "age", "Зарплата": "salary","Члены семьи": "family_numb","Страховые выплаты": "risk"})
data.head(2)

,gender,age,salary,family_numb,risk
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1


Чтобы убедиться, что после запятой нет знака, кроме 0, и мы имеем дело только с целыми числами - просуммируем все значения и тогда увидим все ли числа целые.

In [ ]:
data['age'].sum()

154764.0

In [ ]:
data['age'].value_counts()

19.0    223
25.0    214
31.0    212
26.0    211
22.0    209
27.0    209
32.0    206
28.0    204
29.0    203
30.0    202
23.0    202
21.0    200
20.0    195
36.0    193
33.0    191
24.0    182
35.0    179
34.0    177
37.0    147
39.0    141
38.0    139
41.0    129
18.0    117
40.0    114
42.0     93
43.0     77
44.0     74
45.0     73
46.0     60
47.0     47
49.0     37
50.0     27
48.0     26
52.0     22
51.0     21
53.0     11
55.0      9
54.0      7
56.0      5
59.0      3
60.0      2
58.0      2
57.0      2
62.0      1
65.0      1
61.0      1
Name: age, dtype: int64

признак возраст смело переведем в тип int.

In [ ]:
data['salary'].sum()

199581800.0

зар.плату также можем перевести в признак int, копейки не имеют смысла.


In [ ]:
data['age'] = data['age'].astype('int')

In [ ]:
data['salary'] = data['salary'].astype('int')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
gender         5000 non-null int64
age            5000 non-null int64
salary         5000 non-null int64
family_numb    5000 non-null int64
risk           5000 non-null int64
dtypes: int64(5)
memory usage: 195.4 KB


Далее поищем задваивания и дубликаты.


In [ ]:
data.duplicated().sum()

153

In [ ]:
data.duplicated().sum() * 100 / len(data)

3.06

3% дубликатов, они нам ни к чему, удалим. 3% - капля в море, можем удалить.

In [ ]:
data = data.drop_duplicates().reset_index(drop=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4847 entries, 0 to 4846
Data columns (total 5 columns):
gender         4847 non-null int64
age            4847 non-null int64
salary         4847 non-null int64
family_numb    4847 non-null int64
risk           4847 non-null int64
dtypes: int64(5)
memory usage: 189.5 KB


In [ ]:
data.tail(3)

,gender,age,salary,family_numb,risk
4844,0,20,33900,2,0
4845,1,22,32700,3,0
4846,1,28,40600,1,0


Далее подготовим features and target из исходного датасета

In [ ]:
features = data.drop(['risk'], axis =1)

In [ ]:
features.head(2)

,gender,age,salary,family_numb
0,1,41,49600,1
1,0,46,38000,1


In [ ]:
target = data[['risk']]

In [ ]:
target.head(2)

,risk
0,0
1,1


В этом шаге мы загрузили данные, ознакомились с ними, провели небольшую предобработку: заменили названия столбцов на более удобные для работы (как минимум лишний раз не нужно будет переключать раскладку клавиатуры), перевели 2 признака (возраст и уровень зар.платы) в тип данных int, нашли и удалили дубликаты.

Подготовили обучающие признаки и отделили целевой признак.

## 2. Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** 

Основная наша задача - изменить данные таким образом, чтобы сторонний наблюдатель не получил от них информацию, но эти изменения не повлияли на предсказания линейной регрессии.

Для этого мы попробуем доказать, что при умножении матрицы Х на обратимую матрицу Р, получаем новые значения

(уже поэкспериментировал и увидел, что данные получаются весьма и совершенно иными - как-будто над ними поработала не одна Энигма,  можно ли их вычислить, если не знать матрицы Р?) 

отличные от значений первоначальной матрицы Х, меняются веса w, что потребует вычислить новые веса конечно. НО __предсказание не изменится__! То есть __а__ - предсказание линейной регрессии на первоначальной матрице Х __равно__ предсказаниям переобученной на новых данных после умножения матрицы Х на Р новой линейной регрессии, её предсказание - __а1__.



**Обоснование:**

Мы решили идти от обратного, и говорим, что  $а = а1$ , иначе - не равны.

Пользуясь формулами выше преобразуем данные, вспомнив, что  $Х * (Х)^{-1} = Е$ , где Е - единичная матрица, запомним также следующие свойства -  $E * X = X$ и от перемены мест множителей произведение не меняется.

$$
Xw = XPw1
$$

        небольшое замечание для ясности:
        a = Xw
        a1 = XPw1

$$
X (X^T X)^{-1} X^T y = X P ((X P)^T (X P))^{-1} (X P)^T y
$$

$$
X X^{-1} X^T (X^T)^{-1} y = (X P) (X P)^{-1} (X P)^T ((X P)^T)^{-1} y
$$

$$
E E y = E E y
$$

$$
y = y
$$

В результате мы получили верное равенство. 

## 3. Алгоритм преобразования

**Алгоритм**

Если мы решим создавать свой класс линейной регрессии, то алгоритм будет слледующим:

1. У класса будет 2 функции:
          
        1. fit - обучение, которая принимает тренировочную выборку и целевой признак,
            преобразует тренировочную выборку в подходящую матрицу Х
            и далее расчитывает w и w0.
            
        2. predict - прогноз, которая принимает тестовые данные и строит свой прогноз.
        
2. Обучим алгоритм на исходных данных, получим прогноз и рассчитаем r2_score.

3. Преобразуем исходные данные умножением на обратимую матрицу P.

4. Обучим новую модель на преобразованных данных, получим предикт на преобразованных данных, рассчитаем r2_score.

5. Сравним обе метрики. Должны совпасть.
        
Все расчёты выполняются по формулам из шага №2

## 4. Проверка алгоритма

Для работы мы будем использовать не только свой написанный алгоритм, но и проверим готовым алгоритмом линейной регрессии из библиотеки sklearn.

In [ ]:
class MyLinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        
        self.w = w[1:]
        self.w0 = w[0]
        
    def predict(self, test_features):
        return (test_features.dot(self.w) + self.w0)


Напишем функцию для получения обратимой матрицы P.

        Генерировать случайную обратимую матрицу будем методом - numpy.random.normal()
        
        Необратимые матрицы встречаются редко. 
        Если сгенерировать случайную матрицу функцией numpy.random.normal(),
        вероятность получить необратимую матрицу близка к нулю. Но не нулевая.
        
        Поэтому проверять, что она действительно обратимая, будем методом - numpy.linalg.inv()
        Чтобы проверить - нам нужна будет отдельная функция.
        Если функция необратимая, то в цикле будет сгенерирована новая матрица.

In [ ]:
def get_p(features):
    i = 0
    while i == 0:
        P = np.random.normal(size=(features.shape[1], features.shape[1]))
        i = it_is_good(P)
    return(P)

Функция для проверки обратимости матрицы Р

In [ ]:
def it_is_good(P):
    try:
        np.linalg.inv(P)
        return(1)
    except:
        return(0)

Т.к нам придётся несколько раз обучать модель, но только на разных выборках, то напишем функцию, которая автоматизирует это.

На вход она берет только класс модели, тренировочную выборку, целевой признак и тестовую выборку, возвращает значение r2_score.

            

In [ ]:
def do_work_and_go_to_rest(model_type, features, target):
    model = model_type
    model.fit(features, target)
    predictions = model.predict(features)
    return(r2_score(target, predictions))

__Приступим__

In [ ]:
r2_original = do_work_and_go_to_rest(MyLinearRegression(), features, target)
r2_original

0.4302010046633359

In [ ]:
r2_changed = do_work_and_go_to_rest(MyLinearRegression(), features.dot(get_p(features)), target)
r2_changed

0.4302010046628405

Совпадения от пяти до 14го знака после запятой в зависимости от случайности заполнения матрицы Р. Почти идентичны. Считаю это за успех и практическое доказательство теоретического вывода.

        Далее посмотрим на стандартном алгоритме из библиотеки sklearn

In [ ]:
r2_original = do_work_and_go_to_rest(LinearRegression(), features, target)
r2_original

0.4302010046633359

In [ ]:
r2_changed = do_work_and_go_to_rest(LinearRegression(), features.dot(get_p(features)), target)
r2_changed

0.4302010046633312

Тот же вывод:
    
    Совпадения от пяти до 14го знака после запятой в зависимости от случайности заполнения матрицы Р. Почти идентичны. Считаю это за успех и практическое доказательство теоретического вывода.

                                    ***

__Вывод__

Опираясь на математику - смогли доказать теоретически и подтвердить практически, что качество модели не меняется (принципиально, диагностически не значимо) при изменении данных, главное, чтобы таргет оставался прежним, но это заложено и в формуле.